# Analyzing nitrate data from xylem sap

In [1]:
import pandas as pd
import numpy as np
import pickle
from scipy.stats import t

In [2]:
#Re-open the curve to use to analyze our samples
with open('../02-calibration_curve/calibration_curve.pkl','rb') as f:
    cal_curve=pickle.load(f)

In [3]:
#Re-define this function so we can use it again
def uncertainty(df, cal, sample_col='sample_id',rf_col='rf'):
    """
    Uses the uncertainty in a calibration curve measurement found in Harris'
    Quantitative Chemical Analysis Eighth Edition (Equation 4-27)
    
    Takes as input our calibration curve (saved as a pickle) and a dataframe
    with multiplicate (e.g. triplicate) measurements to give a single
    mean value and its uncertainty

    Assumes dataframe already has a calculated response factor column,
    could probably be generalized to do this as well but seems unnecessary
    """
    m=cal['slope']
    a=cal['intercept']
    s_y=cal['s_yx']
    x_bar=cal['x_mean']
    Sxx=cal['Sxx']
    n_cal=cal['n']
    y_bar=a+m*x_bar
    t_val=t.ppf(0.975,n_cal-2) #Two-sided 95% confidence interval

    results=[]
    for sid, grp in df.groupby(sample_col):
        y_u=grp[rf_col].to_numpy()
        k=len(y_u)
        y_u_bar=np.mean(y_u)
        x_hat=(y_u_bar-a)/m
        s_x=(s_y/m)*np.sqrt((1/k)+(1/n_cal)+((y_u_bar-y_bar)**2)/(m**2*Sxx)) #Equation 4-27
        ci_lo=x_hat-t_val*s_x
        ci_hi=x_hat+t_val*s_x
        results.append({
            "sample":sid,
            "n_rep":k,
            "rf_mean":y_u_bar,
            "conc_mean":x_hat,
            "s_x":s_x,
            "CI_low":ci_lo,
            "CI_high":ci_hi
        })
    return pd.DataFrame(results)

In [4]:
#Read nitrate data
fs_data=pd.read_csv("../01-input_data/fruit_set.csv")
ver_data=pd.read_csv("../01-input_data/veraison.csv")

In [5]:
fs_data.head(5)

,sample,rep,no3_area,istd_area,analysis_day,n_add
0,FS-O-1,1,2.22,10.16,1,0.0
1,FS-O-1,2,2.09,10.71,1,0.0
2,FS-O-1,3,2.24,10.71,1,0.0
3,FS-O-2,1,1.27,13.22,1,0.0
4,FS-O-2,2,1.68,15.58,1,0.0


In [6]:
#Need to remove all entries with 'na'
#fs_data = fs_data.dropna()
#ver_data = ver_data.dropna()
#This is randomly deleting half the dataframe for some reason so just going to delete empty rows by hand

In [7]:
#Let's add response factors
fs_data['rf']=fs_data['no3_area']/fs_data['istd_area']
ver_data['rf']=ver_data['no3_area']/ver_data['istd_area']

In [8]:
fs_data.head(5)

,sample,rep,no3_area,istd_area,analysis_day,n_add,rf
0,FS-O-1,1,2.22,10.16,1,0.0,0.218504
1,FS-O-1,2,2.09,10.71,1,0.0,0.195145
2,FS-O-1,3,2.24,10.71,1,0.0,0.209150
3,FS-O-2,1,1.27,13.22,1,0.0,0.096067
4,FS-O-2,2,1.68,15.58,1,0.0,0.107831


In [9]:
ver_data.head(5)

,sample,rep,no3_area,istd_area,analysis_day,n_add,rf
0,7ppm,1,7.05,12.15,2,NaN,0.580247
1,7ppm,2,5.07,9.14,2,NaN,0.554705
2,7ppm,3,6.95,11.16,2,NaN,0.622760
3,7ppm,1,7.57,14.20,3,NaN,0.533099
4,7ppm,2,6.71,12.72,3,NaN,0.527516


In [10]:
#Let's summarize analytical reps since they're less important for what we're looking at
#We'll need to keep n_add info for downstream analysis
fs_summary=uncertainty(fs_data,cal_curve,sample_col="sample",rf_col="rf")
fs_summary.head(5)

,sample,n_rep,rf_mean,conc_mean,s_x,CI_low,CI_high
0,7ppm,3,0.571157,0.125610,0.033122,0.056920,0.194301
1,80ppm,3,6.725802,1.540749,0.033299,1.471691,1.609808
2,FS-15N-1,3,1.894745,0.429943,0.032616,0.362302,0.497584
3,FS-15N-2,3,1.496906,0.338468,0.032737,0.270575,0.406361
4,FS-15N-3,3,3.035858,0.692320,0.032417,0.625091,0.759548


In [11]:
#Merge metadata
fs_summary_merged=fs_summary.merge(fs_data[['sample','n_add']].drop_duplicates(),on='sample',how='left')

In [12]:
#Repeat for veraison data
ver_summary=uncertainty(ver_data,cal_curve,sample_col="sample",rf_col="rf")
ver_summary.head(5)

,sample,n_rep,rf_mean,conc_mean,s_x,CI_low,CI_high
0,7ppm,9,0.563018,0.123739,0.021800,0.078529,0.168949
1,80ppm,9,6.240292,1.429116,0.021689,1.384135,1.474096
2,V-10N-1,3,0.490702,0.107111,0.033162,0.038338,0.175884
3,V-10N-2,3,0.323880,0.068754,0.033248,-0.000198,0.137706
4,V-10N-3,3,0.820131,0.182857,0.033004,0.114410,0.251304


In [13]:
ver_summary_merged=ver_summary.merge(ver_data[['sample','n_add']].drop_duplicates(),on='sample',how='left')

In [14]:
fs_summary_merged.head(5)

,sample,n_rep,rf_mean,conc_mean,s_x,CI_low,CI_high,n_add
0,7ppm,3,0.571157,0.125610,0.033122,0.056920,0.194301,NaN
1,80ppm,3,6.725802,1.540749,0.033299,1.471691,1.609808,NaN
2,FS-15N-1,3,1.894745,0.429943,0.032616,0.362302,0.497584,3.75
3,FS-15N-2,3,1.496906,0.338468,0.032737,0.270575,0.406361,3.75
4,FS-15N-3,3,3.035858,0.692320,0.032417,0.625091,0.759548,3.75


In [15]:
ver_summary_merged.head(5)

,sample,n_rep,rf_mean,conc_mean,s_x,CI_low,CI_high,n_add
0,7ppm,9,0.563018,0.123739,0.021800,0.078529,0.168949,NaN
1,80ppm,9,6.240292,1.429116,0.021689,1.384135,1.474096,NaN
2,V-10N-1,3,0.490702,0.107111,0.033162,0.038338,0.175884,2.5
3,V-10N-2,3,0.323880,0.068754,0.033248,-0.000198,0.137706,2.5
4,V-10N-3,3,0.820131,0.182857,0.033004,0.114410,0.251304,2.5


In [16]:
fs_summary_merged.to_csv("fs_summary.csv",index=False)
ver_summary_merged.to_csv("ver_summary.csv",index=False)